# TechCraft Gaming PC Production Optimization V3

## Problem Statement
TechCraft builds custom gaming PCs for North American customers. Their flagship product is a high-end gaming rig that customers can customize with different GPUs, cooling systems, and RGB lighting.

### Production Facilities:
- Primary facilities (dedicated lines, must be used):
  * Taiwan facility (T1): 1,800 units/month
  * Mexico facility (M1): 6,000 units/month
  * California facility (C): 1,500 units/month
- Optional facilities (can be repurposed):
  * Taiwan second line (T2): 2,200 units/month
  * Mexico second line (M2): 2,000 units/month

### Fixed Costs for Repurposing:
When repurposing a second production line, TechCraft incurs these fixed costs for tooling and setup:
- Taiwan second line (T2): $21,100
- Mexico second line (M2): $41,700

### Key Constraints:
- All primary production lines must be used
- Only one additional line can be repurposed (due to quality control limitations)
- Cannot exceed production capacity for any line
- Demand must be met exactly (no partial units)
- Distribution to two warehouses: Las Vegas (WH1) and Denver (WH2)

### Expected Demand:
- Las Vegas (WH1): 6,000 units
- Denver (WH2): 5,200 units

### Profit per Unit ($):
| Production Line | Las Vegas (WH1) | Denver (WH2) | Capacity (units/month) |
|----------------|-----------------|---------------|----------------------|
| Taiwan (T1)    | 46              | 49           | 1,800               |
| Mexico (M1)    | 30              | 35           | 6,000               |
| California (C) | 82              | 83           | 1,500               |
| Taiwan 2 (T2)  | 44              | 52           | 2,200               |
| Mexico 2 (M2)  | 86              | 81           | 2,000               |

---

In [ ]:
# Required package installations (uncomment if needed)
# !pip install pulp
# !pip install numpy
# !pip install pandas

import pulp
import numpy as np
import pandas as pd

## Step 1: Define the Linear Programming Problem

In [ ]:
# Create the optimization model
model = pulp.LpProblem("TechCraft_Production_Optimization", pulp.LpMaximize)

# Define production lines and warehouses
production_lines = ['T1', 'M1', 'C', 'T2', 'M2']
warehouses = ['WH1', 'WH2']

# Define profits and capacities
profits = {
    ('T1', 'WH1'): 46, ('T1', 'WH2'): 49,
    ('M1', 'WH1'): 30, ('M1', 'WH2'): 35,
    ('C', 'WH1'): 82, ('C', 'WH2'): 83,
    ('T2', 'WH1'): 44, ('T2', 'WH2'): 52,
    ('M2', 'WH1'): 86, ('M2', 'WH2'): 81
}

capacities = {
    'T1': 1800,
    'M1': 6000,
    'C': 1500,
    'T2': 2200,
    'M2': 2000
}

fixed_costs = {
    'T2': 21100,
    'M2': 41700
}

# Decision variables
vars = pulp.LpVariable.dicts("Units",
                            ((i, j) for i in production_lines for j in warehouses),
                            lowBound=0,
                            cat='Integer')

# Binary variables for optional production lines
use_line = pulp.LpVariable.dicts("UseLine",
                                ['T2', 'M2'],
                                cat='Binary')

# Objective function: profit from sales minus fixed costs for optional lines
model += (pulp.lpSum(vars[i,j] * profits[i,j] for i in production_lines for j in warehouses) -
         pulp.lpSum(use_line[i] * fixed_costs[i] for i in ['T2', 'M2']))

# Demand constraints
model += pulp.lpSum(vars[i,'WH1'] for i in production_lines) == 6000  # Las Vegas demand
model += pulp.lpSum(vars[i,'WH2'] for i in production_lines) == 5200  # Denver demand

# Capacity constraints
for i in production_lines:
    model += pulp.lpSum(vars[i,j] for j in warehouses) <= capacities[i]

# Only one optional line can be used
model += use_line['T2'] + use_line['M2'] <= 1

# Link optional lines to their usage
M = 10000  # Big M constant
for j in warehouses:
    model += vars['T2',j] <= M * use_line['T2']
    model += vars['M2',j] <= M * use_line['M2']

# Solve the model
model.solve()

print(f"Status: {pulp.LpStatus[model.status]}")

## Step 2: Analyze Results

In [ ]:
# Create results dataframe
results = []
for i in production_lines:
    total_production = sum(vars[i,j].value() for j in warehouses)
    if total_production > 0:
        for j in warehouses:
            if vars[i,j].value() > 0:
                results.append({
                    'Production Line': i,
                    'Warehouse': j,
                    'Units': int(vars[i,j].value()),
                    'Capacity': capacities[i],
                    'Capacity Utilization': f"{(total_production/capacities[i])*100:.1f}%",
                    'Profit per Unit': profits[i,j],
                    'Total Profit': int(vars[i,j].value() * profits[i,j])
                })

df_results = pd.DataFrame(results)
print("Production Plan:")
print(df_results)

# Calculate total profit including fixed costs
total_profit = pulp.value(model.objective)
fixed_costs_used = sum(fixed_costs[line] * use_line[line].value() for line in ['T2', 'M2'])

print(f"\nProfit from Production: ${total_profit + fixed_costs_used:,.2f}")
print(f"Fixed Costs: ${fixed_costs_used:,.2f}")
print(f"Net Profit: ${total_profit:,.2f}")

# Show which optional line was used
print("\nOptional Lines Used:")
for line in ['T2', 'M2']:
    if use_line[line].value() == 1:
        print(f"- {line} is active (Fixed Cost: ${fixed_costs[line]:,})")

## Step 3: Verify All Constraints

In [ ]:
# Verify warehouse demands are met
wh_totals = df_results.groupby('Warehouse')['Units'].sum()
print("Warehouse Demand Verification:")
print(f"WH1 (Las Vegas): {wh_totals['WH1']:,} units (Target: 6,000)")
print(f"WH2 (Denver): {wh_totals['WH2']:,} units (Target: 5,200)")

# Verify capacity constraints
print("\nCapacity Constraint Verification:")
for line in production_lines:
    total_prod = sum(vars[line,j].value() for j in warehouses)
    print(f"{line}: {total_prod:,} units used out of {capacities[line]:,} capacity")

# Verify optional line constraint
optional_lines_used = sum(use_line[line].value() for line in ['T2', 'M2'])
print(f"\nOptional lines used: {optional_lines_used} (Maximum: 1)")

## Conclusions

The optimization model has determined:

1. **Optimal Production Plan:** [Will be filled after running]
2. **Maximum Profit:** [Will be filled after running]
3. **Optional Line Usage:** [Will be filled after running]
4. **Capacity Utilization:** [Will be filled after running]

This solution:
- Meets exact demand for both warehouses
- Respects all production capacity constraints
- Uses primary production lines efficiently
- Uses at most one optional line
- Maximizes total profit while considering fixed costs
- Accounts for fixed costs of repurposing production lines